<a href="https://colab.research.google.com/github/hyd3nekosuki/RPDSummerExercise/blob/main/RPDSummerExercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Colaboratoryを活用した拡散計算演習
<p><img alt="Colaboratory logo" height="45px" src="https://www.nagoya-u.ac.jp/common/img/logo.gif" align="left" hspace="10px" vspace="0px"></p> 
<h1> 遠藤 知弘 </h1> 


# [Google Colaboratory](https://colab.research.google.com/)とは？

* Webブラウザから Python を記述、実行できるサービス。以下のような特長あり。
  + 計算環境の構築が不要 
  + 作ったプログラム(.ipynb形式ファイル)を簡単に共有できる
  + GPU への無料アクセス


# Pythonパッケージのインポート(一種のおまじない)

- NumPy: 多次元配列を利用したり、それらの操作のために使用する高度な数学関数ライブラリ群
- matplotlib.pyplot: グラフの描画
- Numba: Pythonプログラムを高速化するために使用


Google Colabにおけるコード実行練習として、**「Shift+Enter」**キーを押して以下のコードを**実行**してみよう。


In [1]:
import numpy as np
import matplotlib.pyplot as plt
from numba import f8, i8, types
from numba.experimental import jitclass
from numba import jit, njit
from numba.typed import List

# 使用する多群断面積データの設定


[KUCA-C架台の2群定数](https://www.kyoto-up.or.jp/book.php?id=1700)
![原子炉物理実験](https://www.kyoto-up.or.jp/upload/book/book_1700_20101008094712.jpg)

[UTR-KINKIの2群定数](https://doi.org/10.1016/0306-4549(91)90048-3)

XSDataクラスの説明については、第52回夏期セミナーテキストのpp.96–100を参考。


以下のプログラムを「Shift+Enter」で実行しておく。

In [2]:
# 多群断面積のクラス
@jitclass([("name", types.string), ("NG", i8),
    ("SigA", f8[:]), ("SigR", f8[:]), ("nuSigF", f8[:]), ("chi", f8[:]), ("D", f8[:]), ("SigTr", f8[:]), ("SigS", f8[:,:]),
    ("SigA0", f8[:]), ("SigR0", f8[:]), ("nuSigF0", f8[:]), ("chi0", f8[:]), ("D0", f8[:]), ("SigTr0", f8[:]), ("SigS0", f8[:,:]) ])
class XSData:
    def __init__(self, name, SigA, nuSigF, chi, D, SigS):
        self.name   = name
        self.SigA   = SigA
        self.nuSigF = nuSigF
        self.chi    = chi
        self.D      = D
        self.SigS   = SigS
        self.NG     = len(self.SigA)
        self.calcSigTrFromD()
        self.balanceSigSgg()
        self.backupOriginalData()

    def calcSigTrFromD(self):
        self.SigTr = 1.0/(3.0*self.D)

    def balanceSigSgg(self):
        # 断面積バランスを満足するように自群散乱を補正
        self.SigS -= np.diag(self.SigA+self.SigS.sum(axis=1) - self.SigTr)
        self.calcSigR()

    def calcSigR(self):
        self.SigR = self.SigA.copy()
        for g in range(self.NG):
            self.SigR[g] += (self.SigS[g,:].sum() - self.SigS[g,g]) # 除去断面積の計算

    def calcKinf(self):
        if self.chi.sum()==0:
            return 0.0
        else:
            flux = np.linalg.solve( (np.diag(self.SigTr) - self.SigS.T), self.chi/self.chi.sum() )
            kinf = self.nuSigF.dot( flux )
            return kinf

    def backupOriginalData(self):
        self.SigA0   = self.SigA.copy()
        self.SigR0   = self.SigR.copy()
        self.nuSigF0 = self.nuSigF.copy()
        self.chi0    = self.chi.copy()
        self.D0      = self.D.copy()
        self.SigTr0  = self.SigTr.copy()
        self.SigS0   = self.SigS.copy()

    def resetData(self):
        self.SigA   = self.SigA0.copy()
        self.SigR   = self.SigR0.copy()
        self.nuSigF = self.nuSigF0.copy()
        self.chi    = self.chi0.copy()
        self.D      = self.D0.copy()
        self.SigTr  = self.SigTr0.copy()
        self.SigS   = self.SigS0.copy()

    def perturbedSigA(self, p=None, g=None): # ad-hoc
        if p==None:
            p=1.0
        if g == None:
            g=0
        self.SigA[g] = p * self.SigA0[g]
        self.SigTr[g] = (p-1) * self.SigA0[g] + self.SigTr0[g] # 拡散係数Dは摂動させない
        self.calcSigR()

## 習うより慣れろ方式で__XSDataクラスのリスト(matlist)__を使ってみよう

XSDataクラスを使用するにあたって、ここまでのプログラムを実行しておく。

あるいは、Google Colabメニュー画面より

**「ランタイム」→「より前のセルを実行」**　あるいは

**「ランタイム」→「すべてのセルを実行」**

と実行してもよい。

In [3]:
matlist = List()
matlist.append(XSData(name="KUCA_C30",  SigA=np.array([0.00320, 0.0930]),    nuSigF=np.array([0.0, 0.168*1.121]),   chi=np.array([1.0,0.0]), D=np.array([1.58, 0.271]),   SigS=np.array([[0.0, 0.0178], [0.0, 0.0]]) ) )
matlist.append(XSData(name="KUCA_C35",  SigA=np.array([0.00286, 0.0850]),    nuSigF=np.array([0.0, 0.149*1.092]),   chi=np.array([1.0,0.0]), D=np.array([1.54, 0.237]),   SigS=np.array([[0.0, 0.0212], [0.0, 0.0]]) ) )
matlist.append(XSData(name="KUCA_C45",  SigA=np.array([0.00237, 0.0724]),    nuSigF=np.array([0.0, 0.121*1.064]),   chi=np.array([1.0,0.0]), D=np.array([1.50, 0.203]),   SigS=np.array([[0.0, 0.0254], [0.0, 0.0]]) ) )
matlist.append(XSData(name="water",     SigA=np.array([0.0,     0.0191]),    nuSigF=np.array([0.0, 0.0]),           chi=np.array([0.0,0.0]), D=np.array([1.41, 0.117]),   SigS=np.array([[0.0, 0.0476], [0.0, 0.0]]) ) )
matlist.append(XSData(name="UTR",       SigA=np.array([1.592e-3, 5.086e-2]), nuSigF=np.array([1.817e-3, 7.335e-2]), chi=np.array([1.0,0.0]), D=np.array([1.458, 0.2144]), SigS=np.array([[0.0, 3.556e-2], [0.0, 0.0]]) ) )
matlist.append(XSData(name="inner_refl",SigA=np.array([1.169e-5, 2.510e-4]), nuSigF=np.array([0.0, 0.0]),           chi=np.array([0.0,0.0]), D=np.array([1.202, 0.8223]), SigS=np.array([[0.0, 2.529e-3], [0.0, 0.0]]) ) )
matlist.append(XSData(name="graphite",  SigA=np.array([3.048e-4, 1.798e-3]), nuSigF=np.array([0.0, 0.0]),           chi=np.array([0.0,0.0]), D=np.array([1.231, 0.6793]), SigS=np.array([[0.0, 3.439e-3], [0.0, 0.0]]) ) )
matlist.append(XSData(name="KUCA_C35PT",SigA=np.array([0.00286, 0.0850+0.01]),nuSigF=np.array([0.0, 0.149*1.092]),   chi=np.array([1.0,0.0]), D=np.array([1.54, 0.237]),   SigS=np.array([[0.0, 0.0212], [0.0, 0.0]]) ) )
matlist.append(XSData(name="waterPT",   SigA=np.array([0.0,     0.0191+0.01]),nuSigF=np.array([0.0, 0.0]),           chi=np.array([0.0,0.0]), D=np.array([1.41, 0.117]),   SigS=np.array([[0.0, 0.0476], [0.0, 0.0]]) ) )


以下のプログラムのうち、配列matidの入力を変更して実行してみよう。

In [4]:
NX = 5

"""
matid = np.full(NX, 3, dtype=int)
matid[0:2] = 1
"""
matid = np.arange(5)

print(matid)
for i in range(NX):
    #print(matlist[matid[i]].name, matlist[matid[i]].SigA)
    print(matlist[matid[i]].name, matlist[matid[i]].calcKinf())

[0 1 2 3 4]


<string>:3: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float64, 1d, A), array(float64, 1d, C))


KUCA_C30 1.7164559139784963
KUCA_C35 1.6866703828663645
KUCA_C45 1.6264707930375202
water 0.0
UTR 1.4293019983078323


# 拡散計算ソルバー用の関数(部品)

各関数の説明については、第52回夏期セミナーテキストのpp.105–108を参考。

これ以降のプログラムを「Shift+Enter」で実行しておく。

## 係数行列Aを計算する関数

1次元平板体系における拡散計算を実施する際に必要となる係数行列(三重対角行列の3つの要素A0, Axm, Axp)を計算する。プログラム中では以下のようにして呼び出して使用する。

    A0, Axm, Axp = calcMatrixA(matlist, matid, dx, bcm, bcp, B2)


In [5]:
# 係数行列Aの設定
@njit(cache=True)
def calcMatrixA(matlist, matid, dx, bcm=None, bcp=None, B2=None):
    NX = len(dx)
    NG = matlist[0].NG

    if bcm == None:
       bcm = "vacuum"
    if bcp == None:
       bcp = "vacuum"
    if B2 == None:
       B2 = 0.0
    
    A0  = np.zeros((NG, NX))
    Axm = np.zeros((NG, NX))
    Axp = np.zeros((NG, NX))

    for g in range(NG):
        A0g  = A0[g,:]
        Axmg = Axm[g,:]
        Axpg = Axp[g,:]
        for i in range(NX):
            mat0 = matlist[matid[i]]

            D0 = mat0.D[g]
            dx0 = dx[i]
            if i==0:
                if bcm == "vacuum":
                    Axmg[i] = -D0/((2.1312*D0 + dx0/2)*dx0)
                elif bcm == "zero":
                    Axmg[i] = -D0/((dx0/2)*dx0)
                else:
                    Axmg[i] = 0.0
            else:
                Dm  = matlist[matid[i-1]].D[g]
                dxm = dx[i-1]
                Axmg[i] = -2*Dm*D0/((Dm*dx0 + D0*dxm)*dx0)

            if i==(NX-1):
                if bcp == "vacuum":
                    Axpg[i] = -D0/((2.1312*D0 + dx0/2)*dx0)
                elif bcp == "zero":
                    Axpg[i] = -D0/((dx0/2)*dx0)
                else:
                    Axpg[i] = 0.0
            else:
                Dp  = matlist[matid[i+1]].D[g]
                dxp = dx[i+1]
                Axpg[i] = -2*D0*Dp/((D0*dxp + Dp*dx0)*dx0)

            #A0g[i] = mat0.SigA[g]+D0*B2 + (mat0.SigS[g,:].sum()-mat0.SigS[g,g])  - (Axmg[i] + Axpg[i])
            A0g[i] = mat0.SigR[g]+D0*B2 - (Axmg[i] + Axpg[i])

    return A0, Axm, Axp

## 核分裂中性子源を計算する関数

核分裂中性子源の空間分布$P(x)$および、その体積積分値を計算する。プログラム中では以下のようにして呼び出して使用する。

    keff = calcProductionRate(flux, P, matlist, matid, dx)


この関数で核分裂中性子源$P(x)$を計算した後、得られた値全体をkeffで割って規格化したければ、以下のように記述すればよい。

    P /= keff

In [6]:
# 核分裂中性子数空間分布と その積分値の計算
@njit(cache=True)
def calcProductionRate(flux, P, matlist, matid, dx):
    NX = matid.shape[0]
    totalFission = 0.0
    for i in range(NX):
        mat0 = matlist[matid[i]]
        fission = (mat0.nuSigF[:]*flux[:, i]).sum() # <- forward
        # fission = mat0.nuSigF[:].dot(flux[:, i]) # <- forward
        P[i] = fission
        totalFission += mat0.chi[:].sum()*fission*dx[i] # <- forward
    return totalFission

## エネルギーg群の中性子源を計算する関数

現時点での核分裂中性子源分布$P(x)$、および中性子束$\phi_{g}(x)$の推定値から、第g群の中性子源Qgを計算する。プログラム中では以下のようにして呼び出して使用する。

    Qg = calcSourceG(g, flux, P, matlist, matid)

In [7]:
# g群の中性子源計算
@njit(cache=True)
def calcSourceG(g, flux, P, matlist, matid):
    NG = flux.shape[0]
    NX = flux.shape[1]
 
    Qg = np.zeros(NX)
    for i in range(NX):
        mat0 = matlist[matid[i]]
        Qg[i] = mat0.chi[g] * P[i] # <- forward
        for gg in range(NG):
            if g != gg:
                Qg[i] += mat0.SigS[gg,g]*flux[gg,i] # <- forward
    return Qg

# 数値解法

各関数の説明については、第52回夏期セミナーテキストのpp.105–111を参考。

これ以降のプログラムを「Shift+Enter」で実行しておく。

## ヤコビ法によるエネルギーg群の中性子束計算

あらかじめ計算済みのg群中性子源Qgに対して、[ヤコビ法](https://mathworld.wolfram.com/JacobiMethod.html)によりg群中性子束を計算する。プログラム中では以下のようにして呼び出して使用する。

    flux[g,:] = diffusionSweepJacobiG(A0[g,:], Axm[g,:], Axp[g,:], flux[g,:], Qg)

なお、上記関数を使用する際には、配列のスライス[g,:]によってg群に対応するA0, Axm, Axpを入力変数として与えて、g群に対応する中性子束flux[g,:]に対して計算結果を代入している。

In [8]:
# ヤコビ法
@njit("(f8[:], f8[:], f8[:], f8[:], f8[:])", cache=True)
def diffusionSweepJacobiG(A0g, Axmg, Axpg, fluxg, Qg):
    NX = fluxg.shape[0]

    fluxgBefore = fluxg.copy()
    fluxgAfter  = fluxg.copy()
    for i in range(NX):
        s = Qg[i]
        if i!=0:
            s -= Axmg[i] * fluxgBefore[i-1]
        if i!=(NX-1):
            s -= Axpg[i] * fluxgBefore[i+1]
        fluxgAfter[i] = s / A0g[i]

    return fluxgAfter

## ガウスザイデル法によるエネルギーg群の中性子束計算

あらかじめ計算済みのg群中性子源Qgに対して、[ガウスザイデル法](https://mathworld.wolfram.com/Gauss-SeidelMethod.html)によりg群中性子束を計算する。プログラム中では以下のようにして呼び出して使用する。

    flux[g,:] =  diffusionSweepGaussSeidelG(A0[g,:], Axm[g,:], Axp[g,:], flux[g,:], Qg)


In [9]:
# ガウスザイデル法
@njit("(f8[:], f8[:], f8[:], f8[:], f8[:])", cache=True)
def diffusionSweepGaussSeidelG(A0g, Axmg, Axpg, fluxg, Qg):
    NX = fluxg.shape[0]

    fluxgAfter  = fluxg.copy()
    for i in range(NX):
        s = Qg[i]
        if i!=0:
            s -= Axmg[i] * fluxgAfter[i-1] # <-
        if i!=(NX-1):
            s -= Axpg[i] * fluxgAfter[i+1] # <-
        fluxgAfter[i] = s / A0g[i]

    return fluxgAfter

## SOR法によるエネルギーg群の中性子束計算

あらかじめ計算済みのg群中性子源Qgに対して、[SOR法](https://mathworld.wolfram.com/SuccessiveOverrelaxationMethod.html)によりg群中性子束を計算する。プログラム中では以下のようにして呼び出して使用する。

    flux[g,:] = diffusionSweepSORG(A0[g,:], Axm[g,:], Axp[g,:], flux[g,:], Qg, omega=1.5)

In [10]:
# 逐次加速緩和法(Successive Over-Relaxation, SOR法)
@njit("(f8[:], f8[:], f8[:], f8[:], f8[:],f8)", cache=True)
def diffusionSweepSORG(A0g, Axmg, Axpg, fluxg, Qg, omega=None):
    if omega is None:
        omega = 1.0
    NX = fluxg.shape[0]

    fluxgAfter  = fluxg.copy()
    for i in range(NX):
        s = Qg[i]
        if i!=0:
            s -= Axmg[i] * fluxgAfter[i-1]
        if i!=(NX-1):
            s -= Axpg[i] * fluxgAfter[i+1]
        fluxgAfter[i] = omega*(s / A0g[i]) + (1-omega)*fluxg[i]

    return fluxgAfter

## 係数行列**A**のLU分解を行う関数

あらかじめ計算済みの係数行列**A**に対して[LU分解](https://mathworld.wolfram.com/LUDecomposition.html)を実施し、行列**L**, **U**の要素を計算する。プログラム中では以下のようにして呼び出して使用する。

    L, U = decomposeLU(A0, Axm, Axp)

なお、各エネルギー群に対してL,Uを計算している点に注意すること。詳細については第52回夏期セミナーテキストのpp.109–110を参考。

In [11]:
# g群のLU分解
@njit("(f8[:],f8[:],f8[:])", cache=True)
def decomposeLUg(A0g, Axmg, Axpg):
    NX = len(A0g)

    Lg =  np.zeros(NX)
    Ug =  np.zeros(NX)
    for i in range(NX):
        if i==0:
            Lg[i] = 0.0
            Ug[i] = A0g[i]
        else:
            Lg[i] = Axmg[i]/Ug[i-1]
            Ug[i] = A0g[i] - Axpg[i-1]*Lg[i]
    return Lg, Ug

In [12]:
# LU分解
@njit("(f8[:,:],f8[:,:],f8[:,:])", cache=True)
def decomposeLU(A0, Axm, Axp):
    NG = A0.shape[0]
    NX = A0.shape[1]

    L =  np.zeros((NG,NX))
    U =  np.zeros((NG,NX))
    
    for g in range(NG):
        L[g,:], U[g,:] = decomposeLUg(A0[g,:], Axm[g,:], Axp[g,:])
    return L, U

## LU分解を利用したエネルギーg群の中性子束計算

あらかじめ計算済みのLU分解後の係数L,Uを用いて、前進消去・後退代入による直接解法によりg群の中性子束を計算する。プログラム中では以下のようにして呼び出して使用する。

    flux[g,:] = diffusionSweepUsingLUG(Axp[g,:], L[g,:], U[g,:], Qg)

なお、上記関数を使用する際には、配列のスライス[g,:]によってg群に対応するAxp, L, Uを入力変数として与えている。

In [13]:
# LU分解を利用したx方向のdiffusion sweep
@njit("(f8[:], f8[:], f8[:], f8[:])", cache=True)
def diffusionSweepUsingLUG(Axpg, Lg, Ug, Qg):
    NX = Axpg.shape[0]
    B = np.zeros(NX)
    BBefore = 0.0
    for i in range(NX):
        B[i] = Qg[i] - BBefore*Lg[i]
        BBefore = B[i]

    fluxAfter = 0.0
    fluxgAfter = np.zeros(NX)
    for i in range(NX-1, -1, -1):
        fluxAfter = (B[i] - Axpg[i]*fluxAfter) / Ug[i]
        fluxgAfter[i] = fluxAfter

    return fluxgAfter

# 演習1 べき乗法による$k_{\mathrm{eff}}$固有値計算ソルバーの作成

第52回夏期セミナーテキストのpp.111–116を参考。

- 1から全て自分で作るのではなく、上で既に定義されている関数(部品)を上手に組み合わせて作成する
  + 物質指定：__matlist__の定義コードを予め実行しておくこと
- 計算速度(外部反復回数が多い/少ない)は拘らず、$k_{\mathrm{eff}}$を求めてみよう
- 上級者は「反復回数が少なく済む」ような計算コードを作ってみよう


## べき乗法作成にあたってfor, if文の練習

In [ ]:
# i=0から10まで、10回繰り返す
N = 100
for i in range(N):
    print(i)
    if i > 10:
        break

for文の簡単な練習問題として、実効増倍率$k_\mathrm{eff}$の体系において$S$個の中性子が核分裂連鎖反応によって何個の子孫中性子を生むのか計算してみよう。

以下のプログラムそのままだと計算が実行できないため、

    children =
の箇所を適切に修正して、プログラムを実行してみよう。

In [ ]:
S = 100    # 源の中性子数
keff = 0.5 # 実効増倍率

sum = 0.0
parent = S
for i in range(1, 101, 1):
    children = 
    sum += children
    #print(children)
    parent = children

print("sum = {}".format(sum))
print("reference = {}".format(S*keff/(1-keff)))

## べき乗法による無限均質体系の$k_{\infty}計算$

べき乗法の原理を理解することができるよう、参考までに、無限均質体系の場合についてべき乗法による無限増倍率の計算プログラム例を以下で示す。

In [ ]:
NG = matlist[0].NG  # エネルギー群数の読み込み
print("energy group = {}".format(NG))

fluxBefere = np.zeros(NG)
mat = matlist[0]

B = np.diag(mat.SigTr) - mat.SigS.T
F = mat.chi.reshape((NG,1)).dot(mat.nuSigF.reshape((1,NG)))

flux = np.ones(NG).reshape((NG,1))
P = F.dot(flux)
kinf = P.sum()
P /= kinf

eps_kinf = 1e-7 # kinfの収束判定基準
eps_flux = 1e-5 # 中性子束の判定基準
kinfBefore = kinf
fluxBefore = flux.copy()
for outer in range(100):
    flux = np.linalg.solve(B, P)
    P = F.dot(flux)
    kinf = P.sum()
    P /= kinf

    if (np.abs(kinf/kinfBefore-1) < eps_kinf ) and \
       (np.abs(flux/fluxBefore - 1).max() < eps_flux):
        break
    kinfBefore = kinf
    fluxBefore = flux.copy()

print("kinf(ref) = {}".format(mat.calcKinf()) )
print("kinf(power method) = {}".format(kinf))

## べき乗法による1次元平板体系における$k_{\mathrm{eff}}計算$

1次元平板体系における実効増倍率$k_\mathrm{eff}$および基本モードの中性子束分布$\phi_{g}(x)$の数値解が得られるように、以下プログラムのうち
    # 1群からN群までの各エネルギー群について以下の処理を繰り返す
        # g群中性子源項を計算
        # g群の内部反復(ヤコビ法、ガウスザイデル法を用いる場合)

    # keffと核分裂中性子源分布を更新
    # 核分裂中性子源分布を規格化
    # 収束判定基準を満足したら外部反復計算を終了
の箇所について、拡散計算ソルバー用の各種関数を組み合わせてプログラムを修正し、実行してみよう。


In [ ]:
NG = matlist[0].NG

# 形状バックリングの設定
y0    = 7.1 * 4
z0    = 57.0
delta = 8.2 # 外挿距離
B2 = (np.pi/(y0+2*delta))**2.0 + (np.pi/(z0+2*delta))**2.0

# 空間メッシュ
NX = 120
matid = np.zeros(NX, dtype=np.int )
matid[0:40]   = 3
matid[40:80]  = 1
matid[-40:]   = 3

dx = np.full(NX, 1.0)
x = dx.cumsum() - dx/2

# 係数行列の設定
A0, Axm, Axp = calcMatrixA(matlist, matid, dx, "vacuum", "vacuum", B2)

flux = np.ones((NG, NX))
P = np.zeros(NX)
keff = calcProductionRate(flux, P, matlist, matid, dx) # keffと核分裂中性子源分布を更新
P /= keff    # 核分裂中性子源分布を規格化

eps_keff = 1e-7
eps_flux = 1e-5
keffBefore = keff
fluxBefore = flux.copy()
for outer in range(1,5):
    # 1群からN群までの各エネルギー群について以下の処理を繰り返す
        # g群中性子源項を計算
        # g群の内部反復(ヤコビ法、ガウスザイデル法を用いる場合)

    # keffと核分裂中性子源分布を更新
    # 核分裂中性子源分布を規格化
    # 収束判定基準を満足したら外部反復計算を終了
    print("outer iteration {0}: keff = {1}".format(outer, keff))


plt.plot(x, flux[0,:], color="blue", linestyle='dashed', label="fast")
plt.plot(x, flux[1,:], color="red", label="thermal" )
plt.xlabel("$x$ (cm)")
plt.ylabel("$\phi$ (a.u.)")
plt.legend()
plt.show()

# 演習2 無限反射体厚さを求めてみよう

第52回夏期セミナーテキストのpp.116–119を参考。

- 演習1で作成した拡散計算ソルバーを__関数calcKeff()__にしてみよう
  + 引数：断面積リスト、物質IDマップ、メッシュ幅、境界条件、y,z方向漏れのバックリング
- 反射体の境界条件を"完全反射(reflective)"と"真空(vacuum)"に変えた際の反応度差$\Delta \rho$がゼロとみなせるような反射体厚さ(メッシュ数)を探索しよう
- 計算コスト削減のため
  + 体系の対称性を利用して「完全反射境界条件」を利用しても良い


## $k_{\mathrm{eff}}$固有値計算の関数作成

まずは、複雑なプログラムを作る前に、演習1で作成したkeff計算プログラムを関数にしてみよう。

例えば、以下のような形で使えるような関数を実装して欲しい。

    keff, flux = calcKeff(matlist, matid, dx, bcp, bcm, B2)

具体的には、以下プログラムのうち

    """
    途中の処理を追加する
    """
の箇所について、演習1で作成したプログラムを流用し、上手く動作するように適宜修正すればよい。


In [ ]:
# keffを求める関数の作成
def calcKeff(matlist, matid, dx, bcm, bcp, B2, eps_keff=None, eps_flux=None, outer_max=None):
    NG = matlist[0].NG
    NX = matid.shape[0]

    if eps_keff == None:
        eps_keff = 1e-7
    if eps_flux == None:
        eps_flux = 1e-5
    if outer_max == None:
        outer_max = 1000

    A0, Axm, Axp = calcMatrixA(matlist, matid, dx, bcm, bcp, B2)
    L, U = decomposeLU(A0, Axm, Axp)

    flux = np.ones((NG, NX))
    P = np.zeros(NX)
    keff = calcProductionRate(flux, P, matlist, matid, dx)
    P /= keff


    """
    途中の処理を追加する
    """

    return keff, flux

## 無限反射体厚さの探索
以上でcalcKeff関数が上手く実装できたら、以下のプログラムのうち

    # 境界条件を変えた時の反応度差がdelta_rhoが十分小さくなったら、反射体メッシュ数の探索を終了する
の箇所に、適切な処理を追加することで、無限厚さとみなすことができる反射体厚さを計算してみよう。

なお、本演習では

 ①反射体の外部境界条件を「真空(vacuum)」とした場合と、
 
 ②反射体の外部境界条件を「完全反射(reflective)」にした場合
 
の2パターンについてkeff計算を実施し、両者の反応度差$|\Delta \rho|<10^{-5}$とみなせるような反射体厚さを探索することで、無限反射体厚さを推定することとする。

In [ ]:
Ncore = 20 
# 形状バックリングの設定
y0    = 7.1 * 4
z0    = 57.0
delta = 8.2 # 外挿距離
B2 = (np.pi/(y0+2*delta))**2.0 + (np.pi/(z0+2*delta))**2.0

delta_rho = np.inf

for Nrefl in range(1,100):
    NX = Ncore + Nrefl
    matid = np.zeros( NX, dtype=np.int )
    matid[0:Ncore] = 1  # <- KUCA C35
    matid[-Nrefl:] = 3  # <- water
    dx = np.full(NX, 1.0)

    keff_r, flux = calcKeff(matlist, matid, dx, "reflective", "reflective", B2)
    keff_v, flux = calcKeff(matlist, matid, dx, "reflective", "vacuum", B2)
    delta_rho = np.abs(1/keff_v - 1/keff_r)
    # 境界条件を変えた時の反応度差がdelta_rhoが十分小さくなったら、反射体メッシュ数の探索を終了する

print("Reflector thickness = {0}: delta_rho = {1}".format(dx[-Nrefl:].sum(), delta_rho) )

x = dx.cumsum() - dx/2
plt.plot(x, flux[0,:], color="blue", linestyle='dashed', label="fast")
plt.plot(x, flux[1,:], color="red", label="thermal" )
plt.xlabel("$x$ (cm)")
plt.ylabel("$\phi$ (cm)")
plt.legend()
plt.show()

参考文献 ["臨界安全ハンドブック第2版", JAERI 1340 (1999).](https://jopss.jaea.go.jp/pdfdata/JAERI-1340.pdf#page=54)

# 演習3 臨界寸法の探索

第52回夏期セミナーテキストのpp.119–120を参考。

- 演習2で作成した__関数calcKeff()__を再活用
- 反射体厚さ(メッシュ数)は演習2で求めた値を入力
- 反応度$\rho \equiv (\frac{k_{\mathrm{eff}} -1}{k_\mathrm{eff}})$がゼロとみなせるような燃料厚さ(メッシュ数)を探索しよう

まず、以下のプログラムにおいて

    Nrefl = ?? # <- 演習2で探索したメッシュ数(反射体厚さ)を指定
の箇所には、演習2で探索した無限反射体厚さとみなせる反射体のメッシュ数を入力すること。

その後、燃料領域のメッシュ数を1メッシュ($\Delta x = 1 \mathrm{cm}$)ずつ増加させ

    # 反応度が最もゼロに近く(keffが最も1に近く)なるような、燃料メッシュ数を調べる
の箇所に適切な処理を追加することで、**炉心半分厚さ(半分メッシュ数) Ncore_crit**の値を推定する。

なお、
    print("Critical thickness = {0}".format( ??? ) )
のについては、対称性を利用している(左側外部境界条件を完全反射として、炉心半分厚さのメッシュ数を求めている)点に注意した上で、臨界寸法を出力すること。

In [ ]:
Nrefl = ?? # <- 演習2で探索したメッシュ数(反射体厚さ)を指定
# 形状バックリングの設定
y0    = 7.1 * 4
z0    = 57.0
delta = 8.2 # 外挿距離
B2 = (np.pi/(y0+2*delta))**2.0 + (np.pi/(z0+2*delta))**2.0

keff_crit = 0.0 
rho_crit = np.inf
Ncore_crit = -1 # 炉心半分厚さ(半分メッシュ数)

for Ncore in range(1,50):
    NX = Ncore + Nrefl
    matid = np.zeros( NX, dtype=np.int )
    matid[0:Ncore] = 1  # <- KUCA C35
    matid[-Nrefl:] = 3  # <- water
    dx = np.full(NX, 1.0)

    keff, flux = calcKeff(matlist, matid, dx, "reflective", "vacuum", B2)
    rho = np.abs(1/keff - 1)
    # 反応度が最もゼロに近く(keffが最も1に近く)なるような、燃料メッシュ数を調べる

print("Critical thickness = {0}".format( ??? ) )
print("keff = {0}".format(keff_crit) )
print("|rho| = {0}".format(rho_crit) )

# 演習4 反射体種類の違いによる結合効果の違い

第52回夏期セミナーテキストのpp.120–121を参考。

- これまでの演習で作成した炉心を、反射体を挟んで2個並べる
- 反射体の種類を変化させると$k_{\mathrm{eff}}$や中性子束はどのように変化するか確認してみよう
  + ただし、まずはy,z方向の形状バックリングはゼロとして調べてみる

なお、以下2つのプログラムにおいて

    Ncore = ??? # 演習3で求めた炉心半分厚さ(メッシュ数)の2倍
    Nrefl = ??? # 演習2で求めた反射体厚さ(メッシュ数)
の2箇所については、これまでの演習2,3で探索した臨界寸法(演習3で求めた炉心半分厚さの2倍)と、無限反射体厚さとみなせるメッシュ数、をそれぞれ入力すること。


In [ ]:
Ncore = ??? # 演習3で求めた炉心半分厚さ(メッシュ数)の2倍
Nrefl = ??? # 演習2で求めた反射体厚さ(メッシュ数)

# 形状バックリングの設定
y0    = 7.1 * 4
z0    = 57.0
delta = 8.2 # 外挿距離
B2 = (np.pi/(y0+2*delta))**2.0 + (np.pi/(z0+2*delta))**2.0
# B2 = 3.105e-3
B2 = 0.0

NX = 1*Ncore + 2* Nrefl
dx = np.full(NX, 1.0)
matid = np.zeros( NX, dtype=np.int )

matid[0:Nrefl] = 3 # <- water 
matid[Nrefl:(Nrefl+Ncore)] = 1 # <- KUCA C35 
matid[-Nrefl:] = 3 # <- water 
keff_lwtr, flux_lwtr = calcKeff(matlist, matid, dx, "vacuum", "vacuum", B2)

matid[0:Nrefl] = 6 # <- graphite
matid[-Nrefl:] = 6 # <- graphite
keff_grph, flux_grph = calcKeff(matlist, matid, dx, "vacuum", "vacuum", B2)

x = dx.cumsum() - dx/2
plt.plot(x, flux_lwtr[0,:], color="blue", linestyle='dashed', label="fast")
plt.plot(x, flux_lwtr[1,:], color="red", label="thermal" )
plt.xlabel("$x$ (cm)")
plt.ylabel("$\phi$ (a.u)")
plt.title("keff = {}".format(keff_lwtr))
plt.legend()
plt.show()

plt.plot(x, flux_grph[0,:], color="blue", linestyle='dashed', label="fast")
plt.plot(x, flux_grph[1,:], color="red", label="thermal" )
plt.xlabel("$x$ (cm)")
plt.ylabel("$\phi$ (a.u)")
plt.title("keff = {}".format(keff_grph))
plt.legend()
plt.show()

In [ ]:
Ncore = ??? # 演習3で求めた炉心半分厚さ(メッシュ数)の2倍
Nrefl = ??? # 演習2で求めた反射体厚さ(メッシュ数)

# 形状バックリングの設定
y0    = 7.1 * 4
z0    = 57.0
delta = 8.2 # 外挿距離
B2 = (np.pi/(y0+2*delta))**2.0 + (np.pi/(z0+2*delta))**2.0
# B2 = 3.105e-3
B2 = 0.0

# 炉心を2個並べる
NX = 2*Ncore + 4* Nrefl
dx = np.full(NX, 1.0)
matid = np.zeros( NX, dtype=np.int )

"""
反射体が軽水の場合についてmatidを設定
"""
keff_lwtr, flux_lwtr = calcKeff(matlist, matid, dx, "vacuum", "vacuum", B2)

"""
反射体を軽水から黒鉛に置き換える
"""
keff_grph, flux_grph = calcKeff(matlist, matid, dx, "vacuum", "vacuum", B2)


x = dx.cumsum() - dx/2
plt.plot(x, flux_lwtr[0,:], color="blue", linestyle='dashed', label="fast")
plt.plot(x, flux_lwtr[1,:], color="red", label="thermal" )
plt.xlabel("$x$ (cm)")
plt.ylabel("$\phi$ (a.u)")
plt.title("keff = {}".format(keff_lwtr))
plt.legend()
plt.show()

plt.plot(x, flux_grph[0,:], color="blue", linestyle='dashed', label="fast")
plt.plot(x, flux_grph[1,:], color="red", label="thermal" )
plt.xlabel("$x$ (cm)")
plt.ylabel("$\phi$ (a.u)")
plt.title("keff = {}".format(keff_grph))
plt.legend()
plt.show()


# 演習5 随伴中性子束を求めてみよう

第52回夏期セミナーテキストのpp.121–127を参考。演習時間の関係上、この演習については、各自の宿題(発展課題)として取り組んでもらいたい。

## 随伴中性子束$\phi^{\dagger}$とは？
- ある場所・あるエネルギーで1個の中性子を体系に投入した時に 「その中性子を起源としてどれだけ核分裂反応が沢山起こるか」を定量化した量
- __インポータンス__とも呼ばれる
- 以下の量を求める際に随伴中性子束$\phi^{\dagger}$が必要
  + 摂動論に基づく微小反応度$\Delta \rho$
  + 任意核データ$\sigma$に対する$k_{\mathrm{eff}}$の変化量(感度係数)
  + 一点炉動特性パラメータ$\Lambda$, $\beta_{\mathrm{eff}}$

## 随伴中性子束の計算方法
- foward計算：__原因__から__結果__を推定する
- adjoint(随伴)計算：__結果__から__原因__を逆推定する
- 決定論的手法の場合、__断面積転置法__により容易に求めることが可能
  * 散乱断面積の2次元行列を__転置(transpose)__する
  * 核分裂スペクトル$\chi_{g}$と生成断面積$\nu \Sigma_{\mathrm{f},g}$を入れ替える





## 随伴核分裂中性子源を計算する関数

随伴核分裂中性子源の空間分布$P^{\dagger}(x)$および、その体積積分値を計算する。プログラム中では以下のようにして呼び出して使用する。

    keff = calcAdjointProductionRate(flux, P, matlist, matid, dx)

**断面積転置法**に基づいて、何と何を転置させればよいのか考えながら、通常(forward)の随伴核分裂中性子源の計算方法を修正してみよう。

In [ ]:
# Adjoint計算用：核分裂中性子数空間分布と その積分値の計算
@njit(cache=True)
def calcAdjointProductionRate(flux, P, matlist, matid):
    NX = matid.shape[0]
    totalFission = 0.0
    for i in range(NX):
        mat0 = matlist[matid[i]]
        fission = (mat0.nuSigF[:]*flux[:, i]).sum() # <- adjointの場合、この行を修正する必要あり
        P[i] = fission
        totalFission += mat0.chi[:].sum()*fission # <- adjointの場合、この行を修正する必要あり
    return totalFission

## エネルギーg群の随伴中性子源を計算する関数

随伴核分裂中性子源分布$P^{\dagger}(x)$、および随伴中性子束$\phi^{\dagger}_{g}(x)$の推定値から、第g群の随伴中性子源Qgを計算する。プログラム中では以下のようにして呼び出して使用する。

    Qg = calcAdjointSourceG(g, flux, P, matlist, matid)

In [ ]:
# Adjoint計算用：g群の中性子源計算
@njit(cache=True)
def calcAdjointSourceG(g, flux, P, matlist, matid):
    NG = flux.shape[0]
    NX = flux.shape[1]
 
    Qg = np.zeros(NX)
    for i in range(NX):
        mat0 = matlist[matid[i]]
        Qg[i] = mat0.chi[g] * P[i] # <- adjointの場合、この行を修正する必要あり
        for gg in range(NG):
            if g != gg:
                Qg[i] += mat0.SigS[gg,g]*flux[gg,i] # <- adjointの場合、この行を修正する必要あり
    return Qg

## $k_{\mathrm{eff}}$随伴固有値計算の関数作成

以上で①随伴核分裂中性子源と②g群の随伴中性子源を計算する関数を上手く作成した上で、$k_{\mathrm{eff}}$随伴固有値計算を実施するための関数を作成してみよう。


In [ ]:
# Adjoint keffを求める関数の作成
def calcAdjointKeff(matlist, matid, dx, bcm, bcp, B2, eps_keff=None, eps_flux=None, outer_max=None):
    NG = matlist[0].NG
    NX = matid.shape[0]

    if eps_keff == None:
        eps_keff = 1e-7
    if eps_flux == None:
        eps_flux = 1e-5
    if outer_max == None:
        outer_max = 1000

    A0, Axm, Axp = calcMatrixA(matlist, matid, dx, bcm, bcp, B2)
    L, U = decomposeLU(A0, Axm, Axp)

    flux = np.ones((NG, NX))
    P = np.zeros(NX)
    keff = calcAdjointProductionRate(flux, P, matlist, matid) # <- adjoint
    P /= keff

    """
    calcKeff()関数の一部をadjoint計算に変更するだけ
    """

    return keff, flux

In [ ]:
Ncore = 19 * 2
Nrefl = 26
# 形状バックリングの設定
y0    = 7.1 * 4
z0    = 57.0
delta = 8.2 # 外挿距離
B2 = (np.pi/(y0+2*delta))**2.0 + (np.pi/(z0+2*delta))**2.0

NX = Nrefl + Ncore + Nrefl
matid = np.full( NX, 1, dtype=np.int )
matid[:Nrefl] = 3
matid[-Nrefl:] = 3
dx = np.full(NX, 1.0)

bcm = "vacuum"
bcp = "vacuum"
keff_fwd, flux_fwd = calcKeff(matlist, matid, dx, bcm, bcp, B2)
keff_adj, flux_adj = calcAdjointKeff(matlist, matid, dx, bcm, bcp, B2)
                    
x = dx.cumsum() - dx/2

plt.plot(x, flux_fwd[0,:], color="blue", linestyle='dashed', label="fast")
plt.plot(x, flux_fwd[1,:], color="red", label="thermal" )
plt.xlabel("$x$ (cm)")
plt.ylabel("$\phi$ (a.u.)")
plt.title("keff(forward) = {0:.5f}".format(keff_fwd))
plt.legend()
plt.show()

plt.plot(x, flux_adj[0,:], color="blue", linestyle='dashed', label="fast")
plt.plot(x, flux_adj[1,:], color="red", label="thermal" )
plt.xlabel("$x$ (cm)")
plt.ylabel("$\phi^{\dagger}$ (a.u)")
plt.title("keff(adjoint) = {0:.5f}".format(keff_adj))
plt.legend()
plt.show()


## 一次摂動論に基づく制御棒価値

$-\Delta \rho \approx  \frac{ \left< \phi^{\dagger} \mathbf{\delta A} {\phi} \right> }{ \left< \phi^{\dagger} \mathbf{F} {\phi} \right> } = \frac{\sum_{g}^{NG} \phi_g^{\dagger} \delta \Sigma_{a,g} \phi_g }{\sum_{g}^{NG}\sum_{g'}^{NG} \phi_{g'}^{\dagger} \chi_{g'} \nu \Sigma_{f,g} \phi_g}$


参考資料：[千葉豪, "炉物理プログラム演習：随伴方程式と摂動計算"](http://roko.eng.hokudai.ac.jp/studentadm/chiba_data/rpgpro/pro_pert3.pdf)

In [ ]:
deltaSigA = np.array( [0.0, 0.01] )
NG = flux_fwd.shape[0]

Ncore = 19 * 2
Nrefl = 26
# 形状バックリングの設定
y0    = 7.1 * 4
z0    = 57.0
delta = 8.2 # 外挿距離
B2 = (np.pi/(y0+2*delta))**2.0 + (np.pi/(z0+2*delta))**2.0
NX = Nrefl + Ncore + Nrefl
matid = np.full( NX, 1, dtype=np.int )
matid[:Nrefl] = 3
matid[-Nrefl:] = 3
dx = np.full(NX, 1.0)

bcm = "vacuum"
bcp = "vacuum"
keff_fwd, flux_fwd = calcKeff(matlist, matid, dx, bcm, bcp, B2)
keff_adj, flux_adj = calcAdjointKeff(matlist, matid, dx, bcm, bcp, B2)

rodworthPT1 = np.zeros(NX) # Note: rodworth is defined as negative reactiveity to be positive
bunbo  = 0.0
for i in range(NX):
    mat0 = matlist[matid[i]]
    dx0 = dx[i]
    for g in range(NG):
        rodworthPT1[i]  += (flux_adj[g,i]*deltaSigA[g]*flux_fwd[g,i])*dx0
        for gg in range(NG):
            bunbo  += (flux_adj[gg,i]*mat0.chi[gg]*mat0.nuSigF[g]*flux_fwd[g,i]) * dx0

rodworthPT1 /= bunbo

rodworthDP = np.zeros(NX)
rodworthPT = np.zeros(NX)
for i in range(NX):
    matid = np.full( NX, 1, dtype=np.int )
    matid[:Nrefl] = 3
    matid[-Nrefl:] = 3
    if i < Nrefl or i>=(Ncore+Nrefl):
        matid[i] = 8
    else:
        matid[i] = 7
    keff_dp, flux_dp = calcKeff(matlist, matid, dx, "vacuum", "vacuum", B2)
    rodworthDP[i] = (1/keff_dp - 1/keff_fwd)

    bunbo  = 0.0
    for g in range(NG):
        rodworthPT[i]  += (flux_adj[g,i]*deltaSigA[g]*flux_dp[g,i])*dx0
    for j in range(NX):
        mat0 = matlist[matid[j]]
        dx0 = dx[j]
        for g in range(NG):
            for gg in range(NG):
                bunbo  += (flux_adj[gg,j]*mat0.chi[gg]*mat0.nuSigF[g]*flux_dp[g,j]) * dx0
    rodworthPT[i] /= bunbo

plt.plot(x, rodworthPT1, label="rod worth(1st-PT)", color="blue", linestyle='dashed')
plt.plot(x, rodworthPT, label="rod worth(PT)", color="purple")
plt.xlabel("$x$ (cm)")
plt.ylabel("$-\Delta \\rho$ (dk/k)")

plt.scatter(x, rodworthDP, label="rod worth(DP)", color="purple", marker= "+")
plt.legend()
plt.show()


## 補足: 摂動論による$k_{\mathrm{eff}}$相対感度係数の算出法

### ある$g$群の巨視的吸収断面積の場合

$\frac{ \Sigma_{\mathrm{a},g} }{k_{\mathrm{eff}}} \frac{\partial k_{\mathrm{eff}} }{ \partial \Sigma_{\mathrm{a},g} } \approx \Sigma_{\mathrm{a},g} \frac{ - \left< \phi^{\dagger} \frac{\mathbf{ \partial A }}{\partial \Sigma_{\mathrm{a
},g} } {\phi} \right> }{ \frac{1}{ k_{\mathrm{eff}} } \left< \phi^{\dagger} \mathbf{F} {\phi} \right> } = -k_{\mathrm{eff}} \frac{ \phi_g^{\dagger} \Sigma_{a,g} \phi_g }{\sum_{g}^{NG}\sum_{g'}^{NG} \phi_{g'}^{\dagger} \chi_{g'} \nu \Sigma_{f,g} \phi_g}$

In [ ]:
mattarget = 1
gtarget   = 1

Ncore = 19 * 2
Nrefl = 26
# 形状バックリングの設定
y0    = 7.1 * 4
z0    = 57.0
delta = 8.2 # 外挿距離
B2 = (np.pi/(y0+2*delta))**2.0 + (np.pi/(z0+2*delta))**2.0
NX = 1*Ncore + 2* Nrefl
dx = np.full(NX, 1.0)
matid = np.zeros( NX, dtype=np.int )

matid[0:Nrefl] = 3 # <- water 
matid[Nrefl:(Nrefl+Ncore)] = 1 # <- KUCA C35 
matid[-Nrefl:] = 3 # <- water 

bcm = "vacuum"
bcp = "vacuum"

# 直接摂動法
delta = 0.001

matlist[mattarget].perturbedSigA(1+delta/2, gtarget)
keffp, fluxp = calcKeff(matlist, matid, dx, bcm, bcp, B2)

matlist[mattarget].perturbedSigA(1-delta/2, gtarget)
keffm, fluxm = calcKeff(matlist, matid, dx, bcm, bcp, B2)

matlist[mattarget].perturbedSigA(1, gtarget)
keff_fwd, flux_fwd = calcKeff(matlist, matid, dx, bcm, bcp, B2)

Sdirect = (keffp - keffm)/keff_fwd /delta


# 一次摂動論
#keff_fwd, flux_fwd = calcKeff(matlist, matid, dx, bcm, bcp, B2)
keff_adj, flux_adj = calcAdjointKeff(matlist, matid, dx, bcm, bcp, B2)

NG = flux.shape[0]
NX = flux.shape[1]
bunbo  = 0.0
bunshi = 0.0
for i in range(NX):
    mat0 = matlist[matid[i]]
    dx0 = dx[i]
    for g in range(NG):
        if matid[i] == mattarget and  g == gtarget:
            bunshi  += (-flux_adj[g,i]*mat0.SigA[g]*flux_fwd[g,i])*dx0
        for gg in range(NG):
            bunbo  += (flux_adj[gg,i]*mat0.chi[gg]*mat0.nuSigF[g]*flux_fwd[g,i]) * dx0

Spt = keff_fwd * bunshi/bunbo

print("direct sensitivity = {:.4f}".format(Sdirect) )
print("perturbation-based sensitivity = {:.4f}".format(Spt) )